# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1769013122000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2025-12-21&endtime=2026-01-20',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.1',
 'count': 8651}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.8,
  'place': '6 km SW of Volcano, Hawaii',
  'time': 1768867035540,
  'updated': 1768867253410,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv74880122',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv74880122&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 50,
  'net': 'hv',
  'code': '74880122',
  'ids': ',hv74880122,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 22,
  'dmin': 0.002906,
  'rms': 0.170000002,
  'gap': 39,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.8 - 6 km SW of Volcano, Hawaii'},
 'geometry': {'type': 'Point',
  'coordinates': [-155.268005371094, 19.3959999084473, 0.720000028610229]},
 'id': 'hv74880122'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.80,"6 km SW of Volcano, Hawaii",1768867035540,1768867253410,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv74880122,",",hv,",",origin,phase-data,",22,0.002906,0.17,39.0,ml,earthquake,"M 1.8 - 6 km SW of Volcano, Hawaii"
1,0.80,"16 km ENE of Fox, Alaska",1768866979411,1768867035871,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak2026biwrof,",",ak,",",origin,phase-data,",10,0.100000,0.60,142.0,ml,earthquake,"M 0.8 - 16 km ENE of Fox, Alaska"
2,1.78,"6 km SW of Volcano, Hawaii",1768866729770,1768866935490,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv74880117,",",hv,",",origin,phase-data,",17,0.001892,0.19,97.0,ml,earthquake,"M 1.8 - 6 km SW of Volcano, Hawaii"
3,1.50,"56 km W of Denali Park, Alaska",1768866633545,1768948652860,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak2026biwmpc,",",ak,",",origin,phase-data,",21,0.400000,1.00,38.0,ml,earthquake,"M 1.5 - 56 km W of Denali Park, Alaska"
4,1.86,"5 km SSW of Volcano, Hawaii",1768866497660,1768866710930,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv74880112,",",hv,",",origin,phase-data,",19,0.002276,0.22,47.0,ml,earthquake,"M 1.9 - 5 km SSW of Volcano, Hawaii"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>